# 14 - CPU: The Complete Processor

## Introduction

The **CPU** brings together the Data Path and Control Unit to implement the **fetch-decode-execute** cycle. This is the heart of our computer!

## Learning Objectives

1. Understand the fetch-decode-execute cycle
2. Integrate Control Unit with Data Path
3. Build a working CPU that can run programs

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))
sys.path.insert(0, str(project_root))

from computer.datapath import DataPath
from computer.control import ControlUnit
from computer.decoder import InstructionDecoder
from computer.clock import Clock
from computer.isa import encode_instruction
from computer import int_to_bits, bits_to_int
from typing import List, Dict

print("Setup complete!")

## Fetch-Decode-Execute Cycle

```mermaid
stateDiagram-v2
    [*] --> FETCH
    FETCH: FETCH<br/>Read PC, Get Instruction
    DECODE: DECODE<br/>Parse Fields
    EXECUTE: EXECUTE<br/>Perform Operation
    WRITEBACK: WRITEBACK<br/>Store Results, Update PC
    
    FETCH --> DECODE
    DECODE --> EXECUTE
    EXECUTE --> WRITEBACK
    WRITEBACK --> FETCH
```

## Exercise 1: CPU

In [ ]:
class CPU:
    """8-bit CPU with fetch-decode-execute cycle."""

    def __init__(self):
        self.datapath = DataPath()
        self.control = ControlUnit()
        self.decoder = InstructionDecoder()
        self.clock = Clock()
        self.halted = False
        self.current_instruction = None

    def reset(self) -> None:
        """Reset CPU to initial state."""
        self.datapath.pc.clock(load=0, load_value=[0] * 8, increment=0, reset=1, clk=1)
        self.control.reset()
        self.halted = False
        self.current_instruction = None

    def fetch(self) -> List[int]:
        """Fetch instruction from memory at PC.

        Returns:
            16-bit instruction
        """
        # YOUR CODE HERE
        pass

    def decode(self, instruction: List[int]) -> Dict:
        """Decode instruction into fields.

        Args:
            instruction: 16-bit instruction

        Returns:
            Decoded fields dictionary
        """
        # YOUR CODE HERE
        pass

    def execute(self, decoded: Dict) -> None:
        """Execute the decoded instruction.

        Args:
            decoded: Decoded instruction fields
        """
        # YOUR CODE HERE
        pass

    def step(self) -> bool:
        """Execute one complete instruction.

        Returns:
            True if CPU should continue, False if halted
        """
        # YOUR CODE HERE
        pass

    def run(self, max_cycles: int = 1000) -> int:
        """Run until HALT or max cycles.

        Returns:
            Number of cycles executed
        """
        cycles = 0
        while cycles < max_cycles and self.step():
            cycles += 1
        return cycles

    def get_state(self) -> Dict:
        """Get current CPU state for debugging."""
        return {
            "pc": self.datapath.get_pc(),
            "flags": self.datapath.flags.copy(),
            "halted": self.halted,
            "cycle": self.clock.cycle,
        }


# Test
cpu = CPU()
print(f"Initial PC: {bits_to_int(cpu.datapath.get_pc())}")
print(f"Halted: {cpu.halted}")

## Running a Simple Program

In [ ]:
def load_program_to_cpu(cpu: CPU, program: List[List[int]]) -> None:
    """Load a program into CPU memory."""
    for i, instr in enumerate(program):
        # Split 16-bit instruction into two bytes
        low_byte = instr[:8]
        high_byte = instr[8:] if len(instr) > 8 else [0] * 8
        addr = i * 2
        cpu.datapath.memory.write(int_to_bits(addr, 8), low_byte, 1)
        cpu.datapath.memory.write(int_to_bits(addr + 1, 8), high_byte, 1)


# Create a simple program: R0 = R1 + R2, then HALT
program = [
    encode_instruction("ADD", rd=0, rs1=1, rs2_imm=2),
    encode_instruction("HALT"),
]

cpu = CPU()

# Set up initial register values
addr_r1 = [1, 0, 0]
addr_r2 = [0, 1, 0]
cpu.datapath.reg_file.write(addr_r1, int_to_bits(25, 8), 1, 0)
cpu.datapath.reg_file.write(addr_r1, int_to_bits(25, 8), 1, 1)
cpu.datapath.reg_file.write(addr_r2, int_to_bits(17, 8), 1, 0)
cpu.datapath.reg_file.write(addr_r2, int_to_bits(17, 8), 1, 1)

# Load program
load_program_to_cpu(cpu, program)

print("Before execution:")
print(f"  R0 = {bits_to_int(cpu.datapath.reg_file.read([0, 0, 0]))}")
print(f"  R1 = {bits_to_int(cpu.datapath.reg_file.read([1, 0, 0]))}")
print(f"  R2 = {bits_to_int(cpu.datapath.reg_file.read([0, 1, 0]))}")

# Run
cycles = cpu.run()

print(f"\nAfter {cycles} cycles:")
print(f"  R0 = {bits_to_int(cpu.datapath.reg_file.read([0, 0, 0]))} (should be 42)")
print(f"  Halted: {cpu.halted}")

## Copy Your Implementation

Once your code works, copy the `CPU` class to `src/computer/cpu.py`

## Validation

In [ ]:
from utils.checker import check

check("cpu")

## Summary

The CPU implements:

| Method | Purpose |
|--------|--------|
| fetch() | Get instruction from memory |
| decode() | Parse instruction fields |
| execute() | Perform the operation |
| step() | One complete instruction |
| run() | Execute until HALT |

### What's Next?

Writing machine code by hand is tedious. Let's build an **Assembler** to convert human-readable assembly language into machine code!